# 12. 使用FP-growth算法来高效发现频繁集

FP-growth 算法基于Apriori构建，但在完成相同任务时采用了一些不同的技术。这里的任务是将数据集存在一个称为FP树的结构之后发现频繁项集或频繁项对，即常在一块出现的元素项的集合FP树。这种做法使得算法的执行速度要快于Apriori。

该算法能更为高效的发现频繁项集，但不能用于发现关联规则。FP-growth算法只扫描数据两次，它发现频繁项集的基本过程如下：

1. 构建FP树
2. 从FP树中挖掘频繁项集

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## 12.1. 创建FP树的数据结构

In [3]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
        
    # 增加count的值
    def inc(self, numOccur):
        self.count += numOccur
    
    # 用于打印出树，便于调试
    def disp(self, ind=1):
        print(" "*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [2]:
# 测试代码
rootNode = treeNode('pyramid', 9, None)
rootNode.children['eye'] = treeNode('eye', 13, None)
rootNode.disp()

  pyramid   9
   eye   13


## 12.2. 构建FP树

除了FP树之外，还需要一个头指针来指向给定类型的第一个实例。利用头指针表可以快速访问FP树中的一个给定类型的所有元素。
![](FP-tree.png)
这里使用一个字典作为数据结构，来保存头部指针，除了存放头部指针，还保存FP树中每类元素的总数

第一遍遍历数据集会获得每个元素项的出现频率。接下来去掉不满足最小支持度的元素项，再下一步构建FP树。

假设有以下交易：

| 交易id | 交易项 |
|:-----:|:-----:|
| 1 | r,z,h,j,p |
| 2 | z,y,x,w,v,u,t,s |
| 3 | z |
| 4 | r,x,n,o,s |
| 5 | y,r,x,z,q,t,p |
| 6 | y,z,x,e,q,s,t,m |

首先遍历一遍数据集，生成头指针表，并排序，把小于最小支持度的元素丢弃

In [34]:
# 生成头部指针表，注意：这里并没有添加头部指针
def genHeadPointerList(dataSet, minsup = 2):
    headdict = {}
    retdict = {}
    for trans in dataSet:
        for i in trans:
            headdict[i] = headdict.get(i,0) + 1
    
    for key,val in headdict.items():
        if val >= minsup:
            retdict[key] = val
            
    return retdict

In [29]:
def loadData(file):
    dataSet = []
    with open(file, 'r') as f:
        for line in f.readlines():
            dataSet.append(line.strip().split(','))
    return dataSet

In [30]:
dataSet = loadData('test')

In [35]:
genHeadPointerList(dataSet, 3)

{'r': 3, 's': 3, 't': 3, 'x': 3, 'y': 3, 'z': 5}

In [38]:
[v[0] for v in sorted(genHeadPointerList(dataSet, 3).items(), key = lambda kv:kv[1], reverse=True)]

['z', 't', 's', 'r', 'y', 'x']

将得到的结果进行排序，除了z之外其他元素的个数相同，这里不妨假设排序的结果为： z,x,y,s,r,t,于是，根据这个结果我们得到了新的交易列表

| 交易id | 交易项 |
|:-----:|:-----:|
| 1 | z,r |
| 2 | z,x,y,s,t |
| 3 | z |
| 4 | x,s,r |
| 5 | z,x,y,r,t |
| 6 | y,x,y,s,t |

接下来我们就可以构建FP树了。下面这张图展示了根据上面表中的数据构建FP树的过程。
![](FP-Tree-gen.png)
依次遍历每一个交易项，从空集开始，向其中不断添加，如果树中已存在现有元素则增加元素的值，否则添加一个新的分支。

In [60]:
def createTree(dataSet, minSup = 1):
    headerTable = genHeadPointerList(dataSet, minSup)
    freqItemSet = set(headerTable)
    
    # 如果没有元素满足要求，则返回None
    if len(freqItemSet) == 0:
        return None, None
    
    # 重新组织字典，为其添加头部指针，初始化为None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
        
    retTree = treeNode("Null Set", 1, None)
    
    # 根据排序后的结果生成新的交易列表，同时生成树
    for trans in dataSet:
        localD = {}
        for item in trans:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0: 
            # 按照出现频率排序，生成新的交易列表
            orderedItems = [v[0] for v in sorted(localD.items(), key = lambda kv:kv[1], reverse=True)]
            print(orderedItems)
            updateTree(orderedItems, retTree, headerTable)
                
    return retTree, headerTable


def updateTree(items, inTree, headerTable):
#     print(items[0])
    if items[0] in inTree.children:
        # 计数增加1
        inTree.children[items[0]].inc(1)
    else:
        # 当前元素没有在树中，建立新子树
        inTree.children[items[0]] = treeNode(items[0], 1, inTree)
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
            
    if len(items) > 1:
        updateTree(items[1:], inTree.children[items[0]], headerTable)
        
        
def updateHeader(nodeToTest, targetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode
    
        

In [61]:
myTree, headerTable = createTree(dataSet, 3)
myTree.disp()

['z', 'r']
['z', 's', 't', 'y', 'x']
['z']
['r', 's', 'x']
['z', 'r', 't', 'y']
['z', 's', 't', 'y', 'x']
  Null Set   1
   r   1
    s   1
     x   1
   z   5
    r   2
     t   1
      y   1
    s   2
     t   2
      y   2
       x   2


## 12.3. 从一棵FP树中挖掘频繁项集jianeryanzhi

从FP树中抽取频繁项集的三个基本步骤如下：
1. 从FP树中获得条件模式基
2. 利用条件模式基，构建一个条件FP树
3. 迭代重复步骤1和2，直到树包含一个元素项为止。

从上一节发现的已经保存在头指针表中的单个频繁元素项开始。对于每一个元素项，获得其对应的条件模式基(conditional pattern base)。条件模式基是以所查找元素项为结尾的路径集合。每一条路径其实都是一条前缀路径(prefix path)。

这里仍以构建FP树的示意图为例，来说明条件模式基。
![](FP-tree.png)

| 频繁项 | 前缀路径 |
|:-----:|:-----:|
| z | {}5 |
| r | {x,s}1, {z,x,y}1, {z}1 |
| x | {z}3, {}1 |
| y | {z,x}3 |
| s | {z,x,y}2, {x}1 |
| t | {z,x,y,s}2, {z,x,y,r}1 |

符号r的前缀路径是{x,s}、{z,x,y}、{z}。每一条前缀路径都与一个计算值关联。该值等于起始元素项的计算值，该值给了每条路径上r的数目。

In [63]:
'''
    参数： basePat: 头指针表中的频繁元素项，如z
          treeNode： 头指针表中元素项对应的节点
'''
def findPrefixPath(basePat, treeNode):
    condPats = {}
    # 通过nodeLike指针找到所有相同的元素
    while treeNode!=None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

'''
    参数： leafNode:   频繁元素节点
          prefixPath: 前缀路径列表
'''
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
        

In [64]:
findPrefixPath('x', headerTable['x'][1])

{frozenset({'s', 't', 'y', 'z'}): 2, frozenset({'r', 's'}): 1}

In [65]:
findPrefixPath('z', headerTable['z'][1])

{}

## 12.4. 创建条件FP树
对于每一个频繁项，都要创建一颗条件FP树

In [66]:
headerTable

{'r': [3, <__main__.treeNode at 0x7f4799476a20>],
 's': [3, <__main__.treeNode at 0x7f4799476160>],
 't': [3, <__main__.treeNode at 0x7f4799476cc0>],
 'x': [3, <__main__.treeNode at 0x7f4799476b00>],
 'y': [3, <__main__.treeNode at 0x7f4799476d68>],
 'z': [5, <__main__.treeNode at 0x7f4799476a90>]}

In [180]:
def mineTree(inTree, headerTable, minSup, prefix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p:p[0])]
    
    for basePat in bigL:
        newFreqSet = prefix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        
        database = []
        #print("condPattBases:", condPattBases)
        # 将condPattBases转成dataSet
        for key,val in condPattBases.items():
            tmplist = [x for x in key]
            for i in range(val):
                database.append(tmplist)

#         print("dataset",database)
        myCondTree, myHead = createTree(database, minSup)
        
        if myHead != None:
            print("conditional tree for:", newFreqSet)
            myCondTree.disp(1)
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)
            

In [188]:
freqItems = []
mineTree(myTree, headerTable, 3, set([]), freqItems)

['z']
['z']
['z']
conditional tree for: {'t'}
  Null Set   1
   z   3
['s']
['s']
['s']
conditional tree for: {'x'}
  Null Set   1
   s   3
['z', 't']
['z', 't']
['z', 't']
conditional tree for: {'y'}
  Null Set   1
   z   3
    t   3
['z']
['z']
['z']
conditional tree for: {'t', 'y'}
  Null Set   1
   z   3


In [189]:
freqItems

[{'r'},
 {'s'},
 {'t'},
 {'t', 'z'},
 {'x'},
 {'s', 'x'},
 {'y'},
 {'t', 'y'},
 {'t', 'y', 'z'},
 {'y', 'z'},
 {'z'}]

## 12.5. 从新闻网站点击流中挖掘

In [203]:
praseData = [line.split() for line in open("kosarak.dat","r").readlines()]
print(praseData[0])

['1', '2', '3']


In [212]:
initSet = genHeadPointerList(praseData)
myFpTree , myHeaderTab = createTree(initSet, 5000)

['1', '3', '6', '8', '0']
['1', '2', '6', '5']
['1', '3', '5', '7']
['1', '3', '4', '6', '9']
['2', '4', '5']
['1', '3', '8', '9', '0']
['1', '7', '8', '9']
['2', '6', '0']
['1', '2', '7', '9']
['1', '4', '9']
['4', '6', '7']
['6', '5', '7']
['1', '4', '6', '9']
['1', '2', '3']
['1', '7', '8', '9']
['2', '3', '7']
['2', '5']
['1', '8', '0']
['1', '4', '8']
['2', '3', '4', '9']
['1', '3', '6', '8', '9']
['3', '4', '9']
['1', '2', '3', '5']
['1', '4', '6', '8']
['1', '4', '8', '9']
['2', '3', '4', '5', '0']
['1', '2', '4', '7']
['1', '6', '5']
['1', '3', '9']
['1', '6', '7', '0']
['1', '3', '9']
['3', '9', '0']
['2', '3', '4', '8']
['1', '2', '7', '0']
['2', '3', '6', '5', '7']
['1', '3', '6', '8']
['2', '3', '6', '9']
['1', '5', '8', '0']
['1', '3', '6', '7']
['1', '3', '6']
['2', '3', '4', '7', '9']
['3', '8', '9', '0']
['2', '3', '7', '9']
['1', '4', '6', '8']
['3', '4', '8']
['1', '3', '4', '8']
['4', '5', '7', '9']
['6', '9', '0']
['1', '2', '3', '8']
['3', '5', '7', '9']
['1', '3',

['1', '3', '6', '0']
['1', '2', '5']
['1', '3', '5', '7', '0']
['3', '5', '8']
['1', '3', '5']
['1', '2', '3']
['1', '2', '4', '6', '0']
['3']
['2', '6', '9']
['2', '3', '5', '8']
['3', '4', '7', '8']
['5', '8', '9']
['1', '2', '4', '5', '7']
['2', '3', '7', '8']
['1', '4', '5', '8']
['2', '4', '7', '8']
['2', '3', '4', '6', '5']
['1', '4', '6', '9', '0']
['1', '4', '7', '8', '9']
['1', '2', '3', '6', '0']
['2', '3', '7', '9']
['3', '6', '5']
['2', '3', '6', '7']
['1', '8', '0']
['2', '4', '6', '7']
['1', '2', '3', '4']
['2', '4', '5', '7']
['1', '3', '6', '7', '9']
['1', '2', '9', '0']
['1', '4', '9']
['1', '2', '5']
['1', '4', '0']
['2', '4']
['1', '2', '7', '8', '9']
['1', '5', '9', '0']
['1', '2', '3', '4']
['1', '4', '6', '7', '8']
['1', '3', '4', '0']
['8', '9']
['1', '2', '3', '5']
['1', '2', '7', '8', '9']
['7', '8']
['2', '6', '8', '9']
['1', '3', '6', '9']
['2', '3', '5', '7']
['1', '2', '0']
['2', '4', '5', '7']
['1', '3', '4', '7']
['2', '3', '5', '8']
['1', '4', '5']
['2',

['2', '3', '0']
['3', '6', '5', '9']
['2', '5', '8']
['1', '0']
['2', '3', '6']
['4', '6', '0']
['1', '2', '4', '6', '8']
['1', '2', '3', '4', '5']
['1', '6', '7', '0']
['3', '4', '6', '7']
['7', '0']
['2', '3', '5']
['2', '3', '4', '9']
['1', '4', '9', '0']
['2', '3', '6', '7']
['2', '3', '5']
['1', '4', '6', '5']
['1', '5', '9']
['1', '2', '4', '0']
['2', '5', '0']
['2', '3', '7', '8']
['3', '4', '7', '0']
['1', '6', '7', '9']
['1', '3', '4', '8']
['2', '5', '7', '0']
['3', '6', '5', '8']
['1', '5', '0']
['1', '2', '8', '0']
['2', '7', '8']
['2', '5', '8', '0']
['1', '3', '0']
['1', '3', '5', '0']
['3', '6', '9', '0']
['6', '7', '8', '9']
['1', '2', '5', '7']
['1', '2', '3', '5', '8']
['4', '7', '8', '0']
['1', '2', '6', '0']
['1', '5', '8', '9']
['2', '3', '6', '7', '9']
['1', '2', '3', '8']
['1', '3', '9']
['1', '2', '7', '9']
['1', '7', '0']
['1', '2', '4', '7', '0']
['1', '3', '6', '5']
['1', '3', '4']
['6', '7', '0']
['3', '9']
['1', '4', '6', '0']
['1', '6', '7', '9', '0']
['2'

['1', '6', '8', '0']
['2', '4', '9']
['1', '2', '3', '4', '8']
['2', '4', '0']
['1', '4', '6']
['1', '3', '6', '9']
['1', '2', '4', '5']
['2', '7', '8']
['1', '3', '5', '8']
['1', '2', '6', '9']
['1', '2', '3', '6', '0']
['2', '3', '5', '7']
['1', '2', '4']
['1', '2', '8', '0']
['1', '7', '8', '0']
['2', '6', '7', '8']
['6', '5', '8']
['4', '6']
['2', '7', '8']
['1', '3', '8', '0']
['1', '6', '5']
['2', '6', '5']
['1', '3', '7', '8']
['2', '6', '8', '0']
['2', '4', '7', '8']
['1', '5', '7', '8']
['2', '3', '5', '9']
['2', '3', '4', '6', '5']
['2', '3', '4', '9']
['1', '2', '3', '9']
['5', '7', '9']
['1', '6', '8']
['1', '3', '4', '6', '8']
['1', '3', '6', '8']
['1', '3', '4', '7']
['1', '3', '4', '7']
['1', '2', '5', '7']
['3', '8', '9']
['3', '4', '5', '0']
['2', '4', '9']
['2', '4', '8', '9']
['2', '4', '6', '9']
['1', '6', '7']
['2', '4', '8', '9']
['6', '5', '8']
['7', '9', '0']
['2', '4', '5', '9']
['2', '3', '5', '0']
['1', '2', '8', '0']
['1', '2', '6', '0']
['1', '2', '6', '7',

['1', '2', '6', '8']
['3', '8', '9', '0']
['1', '3', '4', '7', '8']
['1', '3', '4', '9']
['6', '5', '9']
['1', '3', '4', '5', '9']
['1', '2', '4', '8']
['2', '3', '6', '5']
['1', '4', '5', '9']
['2', '4', '7', '0']
['2', '4', '6', '0']
['2', '3', '5', '7', '9']
['3', '7', '9']
['1', '2', '7', '9']
['3', '6', '5']
['1', '3', '4', '5', '7']
['1', '2', '9']
['1', '2', '7', '8']
['1', '2', '8', '9', '0']
['2', '4', '5', '8']
['1', '2', '5', '8']
['1', '2', '6', '9']
['1', '7', '0']
['2', '6', '5', '0']
['1', '2', '5', '8']
['2', '4', '6', '5', '9']
['2', '3', '4', '7']
['1', '4', '5', '9', '0']
['1', '4', '5', '8']
['3', '4']
['2', '4', '6', '7']
['3', '4', '6', '8']
['7', '8', '9', '0']
['1', '2', '3']
['1', '2', '8', '9']
['2', '8', '0']
['2', '3', '5', '7', '8']
['1', '3', '6', '8']
['3', '6', '8', '0']
['1', '2', '3', '7']
['1', '2', '5']
['1', '2', '4', '6', '8']
['4', '8', '9']
['2', '6', '8']
['6', '5', '7', '9']
['2', '6', '5', '9']
['1', '2', '3', '9']
['1', '6', '5', '8', '0']
['

...

['1', '2', '4', '6']
['1', '2', '3', '4']
['1', '3', '4', '6', '9']
['2', '4', '9']
['1', '4', '8', '9', '0']
['1', '4', '6', '8']
['2', '5', '7', '8']
['1', '2', '3', '6']
['2', '5', '8', '0']
['1', '9', '0']
['7', '8', '9', '0']
['2', '4', '0']
['2', '6', '9']
['1', '3', '4', '0']
['2', '3', '6', '5']
['1', '4', '7']
['1', '2', '4', '5']
['1', '5', '7', '0']
['2', '6', '7']
['2', '3']
['2', '5', '8', '0']
['1', '2', '8', '9']
['1', '2', '3', '8']
['1', '4', '6', '9', '0']
['2', '3', '4', '6']
['1', '2', '6', '5']
['3', '4', '6', '0']
['2', '6', '9']
['1', '3', '6']
['1', '3', '6', '7', '0']
['3', '6', '5', '7']
['2', '4', '7', '9']
['1', '5', '7', '9']
['3', '5', '8', '0']
['1', '3', '5', '7']
['1', '6', '9', '0']
['1', '4', '7', '8']
['2', '3', '4', '8']
['1', '5', '9', '0']
['2', '3', '4']
['3', '5', '9', '0']
['2', '3', '8', '9', '0']
['3', '5', '8']
['3', '5', '0']
['1', '2', '3', '7']
['2', '4', '6', '5']
['1', '2', '6', '5', '8']
['1', '2', '3', '7']
['2', '7', '0']
['1', '2', 

['1', '2', '4', '0']
['1', '3', '9', '0']
['3', '9', '0']
['2', '6', '7', '0']
['2', '3', '4', '0']
['1', '2', '7', '8']
['1', '3', '4', '6', '9']
['3', '4', '6', '5']
['1', '4', '5', '0']
['2', '6', '7', '8']
['4', '6', '5', '0']
['1', '3', '7', '9']
['2', '3', '9', '0']
['1', '2', '4', '8', '0']
['1', '2', '3']
['1', '3', '6', '9']
['1', '5', '0']
['1', '6', '7', '8']
['4', '5', '8', '9']
['1', '2', '6', '8']
['1', '2', '5', '8', '9']
['1', '2', '5', '8']
['2', '3', '4', '5', '0']
['1', '2', '6', '0']
['1', '3', '5', '7', '8']
['2', '4', '5', '0']
['2', '3', '6', '7']
['4', '6', '5']
['2', '3', '4', '5']
['1', '6', '5', '8']
['3', '7', '8']
['1', '3', '6', '7']
['2', '3', '7', '9', '0']
['5', '7', '0']
['1', '3', '4', '7', '8']
['1', '5', '8', '0']
['1', '2', '6']
['3', '5', '8']
['1', '4', '6', '5']
['2', '5', '7', '8']
['1', '8', '9', '0']
['1', '2', '6', '7', '0']
['1', '2', '3', '8']
['1', '2', '6', '5']
['2', '4', '6', '0']
['1', '2', '7', '0']
['1', '2', '5', '7', '9']
['1', '2

['2', '3', '6', '9']
['3', '5', '7', '0']
['1', '3', '4', '7', '8']
['6', '5', '8']
['1', '2', '6', '7']
['2', '4', '5', '8']
['1', '4', '9', '0']
['2', '6', '5', '0']
['2', '3', '4', '7']
['1', '2', '4', '6', '0']
['4', '6', '8']
['1', '3', '4', '6']
['2', '3', '5', '7']
['1', '4', '5', '8', '0']
['1', '2', '3', '4', '0']
['1', '2', '6', '5', '9']
['3', '4', '6', '8', '9']
['2', '3', '6', '7', '8']
['1', '6', '9']
['2', '6', '7', '9']
['3', '0']
['1', '4', '5', '9']
['1', '7', '8', '9']
['2', '3', '5', '7', '9']
['2', '4', '9', '0']
['3', '4', '6', '9']
['1', '4', '6', '9']
['1', '5', '7', '0']
['1', '3', '0']
['1', '3', '6', '8']
['1', '2', '9']
['4', '5', '8', '9']
['1', '2', '5']
['1', '3', '7', '8']
['2', '4', '6', '5', '7']
['1', '2', '4', '6', '7']
['6', '7', '0']
['1', '2', '5', '9']
['3', '6', '7', '9']
['4', '8', '0']
['2', '3']
['2', '4', '5', '8']
['1', '3', '4', '8', '9']
['1', '9']
['1', '2', '9']
['3', '4', '8', '9']
['2', '4', '6', '9']
['2', '3', '9']
['2', '4', '6', '

['1', '2', '4', '5', '9']
['1', '3', '9', '0']
['1', '6', '8', '9']
['3', '6', '9']
['1', '3', '4', '6']
['1', '3', '8', '9']
['2', '3', '6', '9']
['2', '4', '9', '0']
['2', '3', '6', '7']
['3', '6', '9', '0']
['1', '2', '3', '4', '8']
['2', '4', '8', '9']
['2', '3', '6', '8']
['1', '2', '8']
['2', '3', '5']
['3', '4', '6', '5']
['1', '2', '3', '9']
['2', '3', '6', '7']
['1', '2', '3']
['1', '3', '0']
['2', '4', '7', '9', '0']
['2', '3', '4']
['2', '3', '4', '8', '0']
['2', '3', '5', '9']
['2', '4', '6', '5', '9']
['3', '5', '9']
['2', '4', '9', '0']
['6', '5', '9', '0']
['1', '4', '8', '9']
['2', '3', '0']
['2', '3', '8', '9']
['1', '4', '0']
['3', '4', '7']
['1', '3', '8', '9']
['2', '4', '6', '9']
['1', '3', '4', '5', '0']
['1', '2', '8', '9']
['2', '9', '0']
['2', '6', '8', '9', '0']
['2', '3', '0']
['2', '6', '5', '0']
['3', '6', '5', '7']
['1', '2', '6', '9']
['1', '3', '6', '5', '9']
['1', '4', '6', '5', '9']
['1', '4', '5', '0']
['2', '3', '4', '6', '9']
['3', '5', '9']
['2', '

['2', '3', '9']
['1', '4', '8', '9', '0']
['1', '2', '3', '4', '9']
['2', '3', '4']
['2', '5']
['1', '8']
['2', '3', '4', '7']
['1', '2', '8', '0']
['2', '3', '4', '8', '0']
['1', '2', '6', '9']
['1', '2', '4', '8']
['1', '2', '3']
['2', '6', '8']
['3', '4', '8', '9']
['2', '4', '7']
['1', '2', '5', '7']
['2', '3', '4', '9']
['3', '4', '7', '0']
['1', '2', '3', '5', '8']
['1', '4', '5', '7', '9']
['3', '5', '0']
['3', '7']
['3', '6', '5', '9']
['1', '3', '7', '0']
['1', '3', '0']
['2', '6', '5']
['1', '4', '5', '7']
['1', '3', '4', '8']
['2', '3', '6', '0']
['3', '6', '7', '9']
['4', '8', '9']
['4', '5', '7', '9']
['2', '3', '5', '8']
['1', '2', '3', '5']
['3', '5', '8', '9']
['4', '7', '9']
['1', '3', '5', '9', '0']
['1', '2', '3', '5']
['1', '3', '6', '8']
['1', '2', '5', '8', '9']
['3', '4', '7']
['1', '2', '9']
['1', '3']
['1', '2', '5', '9']
['1', '3', '4', '0']
['1', '3', '6', '0']
['4', '5', '7']
['2', '3', '7', '8', '9']
['1', '5', '8', '9']
['2', '4', '6', '0']
['1', '2', '4',

In [218]:
myFreqList = []
mineTree(myFpTree, myHeaderTab, 3500, set([]), myFreqList)

['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']


['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']


['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['3']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['3'

['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['

['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1'

...

['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', 

['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']
['1', '3']

['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['1']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2'

['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']
['1', '2']

['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2', '3']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['2']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3']
['1', '2', '3'

In [219]:
myFreqList

[{'0'},
 {'0', '1'},
 {'0', '2'},
 {'0', '3'},
 {'1'},
 {'2'},
 {'1', '2'},
 {'3'},
 {'1', '3'},
 {'2', '3'},
 {'4'},
 {'1', '4'},
 {'2', '4'},
 {'3', '4'},
 {'5'},
 {'1', '5'},
 {'2', '5'},
 {'3', '5'},
 {'6'},
 {'1', '6'},
 {'2', '6'},
 {'3', '6'},
 {'7'},
 {'1', '7'},
 {'2', '7'},
 {'3', '7'},
 {'8'},
 {'1', '8'},
 {'2', '8'},
 {'3', '8'},
 {'9'},
 {'1', '9'},
 {'2', '9'},
 {'3', '9'}]